In [1]:
import plotly.express as px
import pandas as pd
import json
import shutil
from pathlib import Path
import os

DATAPATH = "./asset/data.json"
RANKPATH = "./asset/rank.txt"

def parse(data, path="./asset/log_4.txt", workers=4):
    year = 1992
    with open(path, "r") as f:
        for line in f.readlines():
            if "time taken" in line:
                entry = {
                    "year": year,
                    "time": int(line.split(" ")[-1]),
                    "workers": workers
                }
                year += 1
                data.append(entry)
    
def rank(path="./asset/log_4.txt", workers=4):
    '''
    Iteration Time: 23
    time taken: 9406
    paper id: 9203203	page rank: 14.87238
    paper id: 9206203	page rank: 14.26908
    paper id: 9206242	page rank: 11.07139
    paper id: 9203206	page rank: 10.25291
    paper id: 9207219	page rank: 10.11316
    '''
    year = 1991
    with open(path, "r") as f, open(RANKPATH, "a") as r:
        r.write(f"\n------ {workers} workers ------\n")
        for line in f.readlines():
            if "Iteration Time" in line:
                year += 1
                r.write(f"Year: {year}\n")
                r.write(line)
            if "time taken" in line:
                r.write(line)
            if "paper id" in line:
                r.write(line)
        

def save(data):
    os.remove(Path(DATAPATH).absolute())
    with open(DATAPATH, mode="w") as f:
        json.dump(data, f)
            
def plot():
    with open(DATAPATH, mode="r") as f:
        benchmarkData = json.load(f)

    benchmarkDataDF = pd.DataFrame(benchmarkData).sort_values(by=['year'])

    fig = px.line(benchmarkDataDF, x='year', y='time', color='workers', markers=True, symbol="workers")
    fig.update_layout(
        title='Spark Pagerank',
        xaxis_title="Year",
        yaxis_title="Time (ms)",
        legend_title="Worker Count",
    )

    fig.show()

In [3]:
def rankit():
    os.remove(Path(RANKPATH).absolute())
    for workers in [1,2,3,4,5]:
        rank(path=f"./asset/log_{workers}.txt", workers=workers)
rankit()

In [4]:
def visualize():
    data = []
    for workers in [1,2,3,4,5]:
        parse(data, path=f"./asset/log_{workers}.txt", workers=workers)
    save(data)
    plot()
        
visualize()